On souhaite étudier les marches aléatoires à pas dans $\{(-1, 0), (0, -1), (1, 0), (0, 1)\}$ qui restent dans le demi-plan supérieur.

In [1]:
@CachedFunction
def count_walks_xy(x, y, n):
    '''
    Count walks that stay in the upper-half plane.
    '''
    if y < 0:
        return 0
    if n == 0 and x == 0 and y == 0:
        return 1
    if n == 0:
        return 0
    return count_walks_xy(x - 1, y, n - 1) + \
           count_walks_xy(x + 1, y, n - 1) + \
           count_walks_xy(x, y - 1, n - 1) + \
           count_walks_xy(x, y + 1, n - 1)


def count_walks(n):
    return sum([count_walks_xy(x, y, n)
                for x in range(-n, n + 1)
                for y in range(n + 1)])

In [2]:
LST = [count_walks(k) for k in range(11)]
print(LST)

[1, 3, 10, 35, 126, 462, 1716, 6435, 24310, 92378, 352716]


In [3]:
OE = oeis(LST)[0]
print(OE)
print(OE.comments())

A001700: a(n) = binomial(2n+1, n+1): number of ways to put n+1 indistinguishable balls into n+1 distinguishable boxes = number of (n+1)-st degree monomials in n+1 variables = number of monotone maps from 1..n+1 to 1..n+1.
 0: To show for example that C(2n+1, n+1) is the number of monotone maps from 1..n + 1 to 1..n + 1, notice that we can describe such a map by a nondecreasing sequence of length n + 1 with entries from 1 to n + 1. The number k of increases in this sequence is anywhere from 0 to n. We can specify these increases by throwing k balls into n+1 boxes, so the total is Sum_{k = 0..n} C((n+1) + k - 1, k) = C(2n+1, n+1).
 1: Also number of ordered partitions (or compositions) of n + 1 into n + 1 parts. E.g., a(2) = 10: 003 030 300 012 021 102 120 210 201 111. - Mambetov Bektur (bektur1987(AT)mail.ru), Apr 17 2003
 2: Also number of walks of length n on square lattice, starting at origin, staying in first and second quadrants. - _David W. Wilson_, May 05 2001. (E.g., for n = 2 t

D'aprè le point 2, notre fonction semble bien compter correctement les marches aléatoires qui restent dans le demi-plan supérieur.

In [4]:
from ore_algebra import *

In [5]:
# Create the shift ore_algebra with rational coefficients
Ind.<n> = PolynomialRing(QQ)
Shift.<Sn> = OreAlgebra(Ind)

In [6]:
# Guess a recurrence satisfied by the sequence (need about 30 terms to find the recurrence)
LST = [count_walks(k) for k in range(50)]
rec = guess(LST, Shift)
print(rec)

(-n - 2)*Sn + 4*n + 6


Sage pense que $w_n$ vérifie la relation :
$w_{n + 1} = \frac{4n + 6}{n + 2}w_n$, qui semble effectivement vérifiée sur les premiers exemples qu'on a calculés explicitement.

In [7]:
LongLST = rec.to_list(LST,301)
print('w_300 :', LongLST[300])

w_300 : 269767020397716795273087713173027318168563459185931167559818600643425130881101115927349629897551350036620178780567668394270379385207482081757077623442427593092384110773141053129456


In [8]:
print('prob of staying in the upper half plane for 300 steps :', (LongLST[300]/4^(300)).n())

prob of staying in the upper half plane for 300 steps : 0.0650116901406073


In [9]:
# Compute the first two asymptotic terms of a basis of the recurrence
print('Une base de l\'espace des solutions de la rec')
print(rec.generalized_series_solutions(2))

Une base de l'espace des solutions de la rec
[4^n*n^(-1/2)*(1 - 5/8*n^(-1) + O(n^(-2)))]


In [10]:
print('Une etude heuristique pour approximer lambda_1')
LongLST = rec.to_list(LST,10001)
print((LongLST[100]/(int(4^100/10))).n())
print((LongLST[10000]/(int(4^10000/100))).n())

Une etude heuristique pour approximer lambda_1
1.12139052285748
1.12830864983222


##### La partie qui me concerne commence vraiment maintenant

In [11]:
# Create the differential algebra to encode linear differential equation
Pols.<z> = PolynomialRing(QQ)
Diff.<Dz> = OreAlgebra(Pols)

In [12]:
# Guess an annihilating linear differential equation for generating function
diffWalk = guess(LST,Diff)
print(diffWalk)

(4*z^2 - z)*Dz^2 + (14*z - 2)*Dz + 6


In [13]:
# Converting from the differential equation to a recurrence 
# gives rec (up to a left multiple)
print(Diff(diffWalk).to_S(Shift))

(-n^2 - 3*n - 2)*Sn + 4*n^2 + 10*n + 6


C'est $(n + 1) * rec$

La théorie implique que toute singularité d'une solution de `diffWalk` doit être solution de $4z^2 - z$, autrement dit est en $0$ ou $\frac{1}{4}$.
Comme $w_n \leq 4^n$ pour tout $n$, on sait que la série est convergente au voisinage de l'origine. La seule singularité possible (si la série en a une) est donc en $\frac{1}{4}$.

In [14]:
diffWalk.local_basis_expansions(0, order = 4)

[z^(-1), 1 + 3*z + 10*z^2]

On nomme $A_1 (z) = z^{-1}$ et $A_2 (z) = 1 + 3z + 10z^2 + \dots$.

Comme $W$ est convergente au voisinage de 0, on a en fait $W = A_2$.

In [15]:
diffWalk.local_basis_expansions(1/4, order = 4)

[(z - 1/4)^(-1/2) - 4*(z - 1/4)^(1/2) + 16*(z - 1/4)^(3/2) - 64*(z - 1/4)^(5/2),
 1 - 4*(z - 1/4) + 16*(z - 1/4)^2 - 64*(z - 1/4)^3]

On pose

$B_1 (z) = {\left(z - \frac{1}{4}\right)}^{- 1 / 2} - 4 {\left(z - \frac{1}{4}\right)}^{1 / 2} + \dots$

$B_2 (z) = 1 - 4 (z - \frac{1}{4}) + \dots$.

Il faut trouver la matrice de passage de $(A)$ vers $(B)$.

In [16]:
# Represent W in terms of the A_j basis
ini = [0, 1]

# Find numeric coefficients of W in the B_j basis
trans_mat = diffWalk.numerical_transition_matrix([0, 1/4])
bas = trans_mat * vector(ini)
print(trans_mat)
print(bas)

[                [+/- 1.54e-22]*I [1.0000000000000000 +/- 4e-22]*I]
[  [4.0000000000000000 +/- 1e-21]  [-2.0000000000000000 +/- 3e-21]]
([1.0000000000000000 +/- 6.94e-18]*I, [-2.0000000000000000 +/- 1.39e-17])


On nomme $\rho_1$ et $\rho_2$ ces constantes. Les bornes sur l'approximation permettent de déduire que $\rho_1 \neq 0$ (et suggèrent fortement que $\rho_1 = i$ et $\rho_2 = -2$).

En particulier, $\frac{1}{4}$ est bien la singularité dominante de $W$.

$W(z) = \rho_1 B_1 + \rho_2 B_2$

Or en $0$,
$
[z^n]B_1 (z)
\sim [z^n]{\left(z - \frac{1}{4}\right)}^{- 1 / 2}
= [z^n] 2i {\left(1 - 4z\right)}^{- 1 / 2}
= [z^n] 2i \left( 1 + 2z + 6z^2 + \dots + \right)
$

Donc $w_n \sim 2 {2n \choose n} \sim \frac{2}{\sqrt{\pi}}\frac{4^n}{\sqrt{n}}$

In [17]:
from math import sqrt, pi


def f(n):
    return 2 / sqrt(pi) * (4 ** n / sqrt(n))

print(f(10) / LongLST[10])
print(f(50) / LongLST[50])
print(f(100) / LongLST[100])
print(f(200) / LongLST[200])
print(f(500) / LongLST[500])

1.0607909645261389
1.0124288589669703
1.0062321235070055
1.0031205193768193
1.0012492819913577


Il semble qu'on a bien déterminé le premier terme du développement asymptotique de $w_n$.